Fine tuning Mongolian BERT for eduge classification. cased BERT base 32K 512

In [1]:
import os
from os.path import exists, join, basename, splitext
import sys

def download_from_google_drive(file_id, file_name):
  # download a file from the Google Drive link
  !rm -f ./cookie
  !curl -c ./cookie -s -L "https://drive.google.com/uc?export=download&id=$file_id" > /dev/null
  confirm_text = !awk '/download/ {print $NF}' ./cookie
  confirm_text = confirm_text[0]
  !curl -Lb ./cookie "https://drive.google.com/uc?export=download&confirm=$confirm_text&id=$file_id" -o $file_name
  
# download a pre-trained model
model_path = 'cased_bert_base'
if not exists(model_path):
  download_from_google_drive('1MOZUKppfX45BEh7nxQ5AvzK-8wIUITr8', '%s.zip' % model_path)
  zip_file = model_path+".zip"
  !unzip $zip_file
  sys.path.append(model_path)

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   388    0   388    0     0   4217      0 --:--:-- --:--:-- --:--:--  4217
100 1166M    0 1166M    0     0   105M      0 --:--:--  0:00:11 --:--:--  107M
Archive:  cased_bert_base.zip
   creating: cased_bert_base/
  inflating: cased_bert_base/eval_results.txt  
  inflating: cased_bert_base/mn_cased.model  
  inflating: cased_bert_base/checkpoint  
  inflating: cased_bert_base/model.ckpt-4000000.data-00000-of-00001  
  inflating: cased_bert_base/model.ckpt-4000000.index  
  inflating: cased_bert_base/model.ckpt-4000000.meta  
  inflating: cased_bert_base/mn_cased.vocab  
  inflating: cased_bert_base/tokenization_sentencepiece.py  
  inflating: cased_bert_base/bert_config.json  


In [2]:
import os
import sys
from os.path import exists, join, basename, splitext

project_path = 'mongolian-bert'
  
sys.path.append(project_path)

if not exists(project_path):
  print("checking out")
  !git clone -q --recursive https://github.com/tugstugi/mongolian-bert.git

checking out


In [3]:
!pip install -r ./mongolian-bert/requirements.txt

    100% |████████████████████████████████| 1.0MB 20.8MB/s 
    100% |████████████████████████████████| 81kB 24.3MB/s 
    100% |████████████████████████████████| 112kB 31.3MB/s 
  Stored in directory: /root/.cache/pip/wheels/84/11/01/951369cbbf8f96878786a1f4da68bd7ac19a5d945b38e03d54
Successfully built EbookLib


In [0]:
import datetime
import json
import os
import pprint
import random
import string
import sys
import tensorflow as tf


In [5]:
from tokenization_sentencepiece import FullTokenizer
tokenizer_model_path = model_path
tokenizer  = FullTokenizer(model_file=join(tokenizer_model_path, 'mn_cased.model'), vocab_file=join(tokenizer_model_path, 'mn_cased.vocab'), do_lower_case=False)

Loaded a trained SentencePiece model.


In [0]:
import datetime
import json
import os
import pprint
import random
import string
import sys
import tensorflow as tf

In [7]:
LEN                 = 512
BERT_PRETRAINED_DIR = "./cased_bert_base"
DATASET             = 'eduge'
OUTPUT_DIR          = './{}_{}_cased'.format(DATASET, LEN)
os.makedirs(OUTPUT_DIR)
print(OUTPUT_DIR)

./eduge_512_cased


In [0]:
sys.path.append("./mongolian-bert/bert")
import modeling
import optimization
import run_classifier
import tokenization

In [0]:
MODEL_CHECKPOINT = "model.ckpt-4000000"
VOCAB_FILE       = os.path.join(model_path, 'mn_cased.vocab')
CONFIG_FILE      = os.path.join(model_path, 'bert_config.json')
INIT_CHECKPOINT  = os.path.join(BERT_PRETRAINED_DIR, MODEL_CHECKPOINT)
#INIT_CHECKPOINT  = "/content/cased_bert_base/model.ckpt-4000000"
DO_LOWER_CASE    = False

In [10]:
!pwd

/content


In [11]:
import os
if not os.path.exists("eduge.csv.gz"):
  !wget https://github.com/tugstugi/mongolian-nlp/raw/master/datasets/eduge.csv.gz
  !gunzip eduge.csv.gz

--2019-04-11 01:04:21--  https://github.com/tugstugi/mongolian-nlp/raw/master/datasets/eduge.csv.gz
Resolving github.com (github.com)... 192.30.253.112, 192.30.253.113
Connecting to github.com (github.com)|192.30.253.112|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/tugstugi/mongolian-nlp/master/datasets/eduge.csv.gz [following]
--2019-04-11 01:04:21--  https://raw.githubusercontent.com/tugstugi/mongolian-nlp/master/datasets/eduge.csv.gz
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 73951267 (71M) [application/octet-stream]
Saving to: ‘eduge.csv.gz’

eduge.csv.gz        100%[===================>]  70.53M   166MB/s    in 0.4s    

2019-04-11 01:04:22 (166 MB/s) - ‘eduge.csv.gz’ saved [73951267/73

In [12]:
import pandas as pd
df = pd.read_csv("eduge.csv")
df = df.rename(columns=lambda x: x.strip())
df.columns.values
labels = df['label'].unique().tolist()
labels

['урлаг соёл',
 'эдийн засаг',
 'эрүүл мэнд',
 'хууль',
 'улс төр',
 'спорт',
 'технологи',
 'боловсрол',
 'байгал орчин']

In [0]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(df, test_size=0.1, random_state=999, stratify=df['label'])

In [0]:
DATA_COLUMN  = 'news'
LABEL_COLUMN = 'label'
label_list   = labels

In [0]:
train_InputExamples = train.apply(lambda x: run_classifier.InputExample(guid=None,
                                                                   text_a = x[DATA_COLUMN], 
                                                                   text_b = None, 
                                                                   label  = x[LABEL_COLUMN]), axis = 1)
test_InputExamples  = test.apply(lambda x: run_classifier.InputExample(guid=None, 
                                                                   text_a = x[DATA_COLUMN], 
                                                                   text_b = None, 
                                                                   label  = x[LABEL_COLUMN]), axis = 1)

In [0]:
LEARNING_RATE          = 2e-5
NUM_TRAIN_EPOCHS       = 4.0
WARMUP_PROPORTION      = 0.1
SAVE_CHECKPOINTS_STEPS = 500
SAVE_SUMMARY_STEPS     = 100
TRAIN_BATCH_SIZE       = 4 #256
EVAL_BATCH_SIZE        = 4 #256
PREDICT_BATCH_SIZE     = 4
ITERATIONS_PER_LOOP    = 100
NUM_TPU_CORES          = 8
MAX_SEQ_LENGTH         = LEN

should_train           = True

In [17]:
train_features = run_classifier.convert_examples_to_features(train_InputExamples, label_list, MAX_SEQ_LENGTH, tokenizer)
test_features  = run_classifier.convert_examples_to_features(test_InputExamples , label_list, MAX_SEQ_LENGTH, tokenizer)

num_train_steps  = int(len(train_features) / TRAIN_BATCH_SIZE * NUM_TRAIN_EPOCHS)
num_test_steps   = int(len(test_features) / EVAL_BATCH_SIZE)
num_warmup_steps = int(num_train_steps * WARMUP_PROPORTION)

INFO:tensorflow:Writing example 0 of 68094
INFO:tensorflow:*** Example ***
INFO:tensorflow:guid: None
INFO:tensorflow:tokens: [CLS] ▁Засгийн ▁газрын ▁гишүүн , ▁УИХ - ын ▁гишүүн ▁Х . Баттулга ▁сайд аасаа ▁тат зал зах ▁хүсэлтээ ▁өнгөрсөн ▁долоо ▁хоногт ▁өгсөн ▁билээ . ▁Тэрээр ▁сайд аасаа ▁татгалзах ▁хүсэлт дээ , ▁Засгийн ▁газрын ▁гишүүний ▁ажлаа ▁өгч , ▁УИХ - ын ▁гишүүний ▁үүргээ ▁үргэлж үүлэх ээр ▁болсноо ▁дурьдсан ▁бөгөөд ▁цаашид ▁үйлдвэрлэл , ▁хөдөө ▁аж ▁ахуйн ▁салбарт ▁эхлүүлсэн ▁ажлаа ▁үргэлж үүлэхэд ▁анхаарч ▁ажиллана ▁гэдгээ ▁мэдэгдсэн . ▁Өнгөрсөн ▁долоо ▁хоногт ▁Ерөнхийлөгчөөс ▁өргөн ▁барьсан ▁Засгийн ▁газрын ▁гишүүний ▁үүргийг ▁давхар ▁хашдаг ▁УИХ - ын ▁гишүүдийг ▁" дан ▁дээлтэй " ▁болгох ▁хуулийн ▁төслийг ▁гурван ▁сарын ▁хугацаанд ▁хэлэлцэж , ▁унагасан . ▁УИХ - ын ▁гишүүдийг ▁" давхар ▁дээл "- ийг ▁тайлах ▁хуулийн ▁төслийг ▁2014 ▁оноос ▁хэрэгжүүлэх ▁ёстой ▁гэдэг ▁дээр ▁Х . Баттулга ▁сайд ▁тууштай ▁байсан ▁бөгөөд ▁өгсөн ▁саналаа ▁бататгаж , ▁өргөдлөө ▁өгсөн ▁гэж ▁зарим ▁хүмүүс ▁

In [18]:
print(CONFIG_FILE)

cased_bert_base/bert_config.json


In [0]:
train_input_fn = run_classifier.input_fn_builder(
    features       = train_features,
    seq_length     = MAX_SEQ_LENGTH,
    is_training    = True,
    drop_remainder = True)

In [20]:
run_config = tf.contrib.tpu.RunConfig(
    model_dir = OUTPUT_DIR,
    save_checkpoints_steps = SAVE_CHECKPOINTS_STEPS,
    tpu_config = tf.contrib.tpu.TPUConfig(
        iterations_per_loop         = ITERATIONS_PER_LOOP,
        num_shards                  = NUM_TPU_CORES,
        per_host_input_for_training = tf.contrib.tpu.InputPipelineConfig.PER_HOST_V2))

model_fn = run_classifier.model_fn_builder(
    bert_config      = modeling.BertConfig.from_json_file(CONFIG_FILE),
    num_labels       = len(label_list),
    init_checkpoint  = INIT_CHECKPOINT,
    learning_rate    = LEARNING_RATE,
    num_train_steps  = num_train_steps,
    num_warmup_steps = num_warmup_steps,
    use_tpu          = False,
    use_one_hot_embeddings = True)

estimator = tf.contrib.tpu.TPUEstimator(
    use_tpu            = False,
    eval_on_tpu        = False,
    model_fn           = model_fn,
    config             = run_config,
    train_batch_size   = TRAIN_BATCH_SIZE,
    eval_batch_size    = EVAL_BATCH_SIZE,
    predict_batch_size = PREDICT_BATCH_SIZE)



For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

INFO:tensorflow:Using config: {'_model_dir': './eduge_512_cased', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 500, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': None, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fdc222e9d30>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief

In [21]:
from datetime import datetime

if should_train:
  print("Beginning Training!")
  current_time = datetime.now()
  estimator.train(input_fn=train_input_fn, max_steps=num_train_steps)
  print("Training took time ", datetime.now() - current_time)

Beginning Training!
Instructions for updating:
Colocations handled automatically by placer.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Running train on CPU
INFO:tensorflow:*** Features ***
INFO:tensorflow:  name = input_ids, shape = (4, 512)
INFO:tensorflow:  name = input_mask, shape = (4, 512)
INFO:tensorflow:  name = label_ids, shape = (4,)
INFO:tensorflow:  name = segment_ids, shape = (4, 512)
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use keras.layers.dense instead.
INFO:tensorflow:**** Trainable Variables ****
INFO:tensorflow:  name = bert/embeddings/word_embeddings:0, shape = (32000, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/embeddings/token_type_embeddings:0, shape = (2, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/embeddings/position_embeddings:0, shape = (512, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/embeddings/LayerNorm/beta:0, shape = (768,

KeyboardInterrupt: ignored

In [22]:
!ls -alh

total 1.5G
drwxr-xr-x 1 root root 4.0K Apr 11 01:04 .
drwxr-xr-x 1 root root 4.0K Apr 11 01:02 ..
drwxr-xr-x 3 root root 4.0K Apr 11 01:04 cased_bert_base
-rw-r--r-- 1 root root 1.2G Apr 11 01:03 cased_bert_base.zip
drwxr-xr-x 4 root root 4.0K Apr  4 20:19 .config
-rw-r--r-- 1 root root  492 Apr 11 01:03 cookie
drwxr-xr-x 2 root root 4.0K Apr 11 02:57 eduge_512_cased
-rw-r--r-- 1 root root 306M Apr 11 01:04 eduge.csv
drwxr-xr-x 9 root root 4.0K Apr 11 01:04 mongolian-bert
drwxr-xr-x 1 root root 4.0K Apr  4 20:20 sample_data


In [0]:
test_input_fn = run_classifier.input_fn_builder(
    features       = test_features,
    seq_length     = MAX_SEQ_LENGTH,
    is_training    = False,
    drop_remainder = True)

In [24]:
estimator.evaluate(input_fn = test_input_fn, steps=num_test_steps)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Running eval on CPU
INFO:tensorflow:*** Features ***
INFO:tensorflow:  name = input_ids, shape = (4, 512)
INFO:tensorflow:  name = input_mask, shape = (4, 512)
INFO:tensorflow:  name = label_ids, shape = (4,)
INFO:tensorflow:  name = segment_ids, shape = (4, 512)
INFO:tensorflow:**** Trainable Variables ****
INFO:tensorflow:  name = bert/embeddings/word_embeddings:0, shape = (32000, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/embeddings/token_type_embeddings:0, shape = (2, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/embeddings/position_embeddings:0, shape = (512, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/embeddings/LayerNorm/beta:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/embeddings/LayerNorm/gamma:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_0/attention/self/query/kernel:0, shape = (768, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/la

{'eval_accuracy': 0.89939183,
 'eval_loss': 0.44815582,
 'global_step': 5500,
 'loss': 0.44815582}

In [0]:
def getPrediction(in_sentences):
  input_examples   = [run_classifier.InputExample(guid=None, text_a = x, text_b = None, label = "урлаг соёл") for x in in_sentences]
  input_features   = run_classifier.convert_examples_to_features(input_examples, label_list, MAX_SEQ_LENGTH, tokenizer)
  predict_input_fn = run_classifier.input_fn_builder(features=input_features, seq_length=MAX_SEQ_LENGTH, is_training=False, drop_remainder=True)
  predictions      = estimator.predict(predict_input_fn)
  return predictions

In [0]:
pred_sentences = [
  "Монгол улсын уул уурхайн гарц энэ жилээс эхлэн нэмэгдэж дэлхийн эдийн засаг сэргэсний азаар эдийн засагт таатай сайхан нөхцөл байдал үүсээд байна.",
  "Хиймэл оюун болоод дуу хоолой таних систем ухаалаг туслагчийн үр нөлөөгөөр ухаалаг утас хэрэглэгчдийн тоо эрс нэмэгджээ.",
  "Гурвыг харьцах гурав заалны сагсан бөмбөгийн спортод манай тамирчид одтой байна.",
  "Ханиад томуунаас урьдчилан сэргийлэхийн тулд биеэ чийрэгжүүлэх шаардлагатай."    
]

In [27]:
import numpy as np
predictions = getPrediction(pred_sentences)
for (sent, pred) in zip(pred_sentences, predictions):
  print("==============SENTENCE===============")
  print(sent)
  probabilities = pred["probabilities"]
  softmax       = np.exp(probabilities)/sum(np.exp(probabilities))
  label_index   = np.argmax(softmax)
  print(label_list[label_index])

INFO:tensorflow:Writing example 0 of 4
INFO:tensorflow:*** Example ***
INFO:tensorflow:guid: None
INFO:tensorflow:tokens: [CLS] ▁Монгол ▁улсын ▁уул ▁уурхайн ▁гарц ▁энэ ▁жилээс ▁эхлэн ▁нэмэгдэж ▁дэлхийн ▁эдийн ▁засаг ▁сэргэ сний ▁азаар ▁эдийн ▁засагт ▁таатай ▁сайхан ▁нөхцөл ▁байдал ▁үүсээд ▁байна . [SEP]
INFO:tensorflow:input_ids: 4 29 44 580 682 1911 22 2643 831 1219 272 88 335 11703 2339 21142 88 1550 1300 278 368 205 4346 11 7 5 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0